# Archimedes Chess AI - Google Colab Setup

Dieses Notebook führt Sie durch die Installation und das Training der Archimedes Schach-KI in Google Colab.

## Voraussetzungen

1. **GPU aktivieren**: `Laufzeit` → `Laufzeittyp ändern` → `Hardwarebeschleuniger: GPU`
2. **Repository klonen oder hochladen** (siehe nächste Zelle)

## Übersicht

- **Zelle 1**: Installation aller Dependencies
- **Zelle 2**: Repository Setup (Git Clone oder Upload)
- **Zelle 3**: System-Check (GPU, CUDA)
- **Zelle 4**: Optional - System Benchmark
- **Zelle 5**: Training starten

In [ ]:
# Zelle 1: Installation aller Dependencies
# Diese Zelle installiert alle benötigten Pakete für Archimedes

print("📦 Installiere Dependencies...")

# Basis-Dependencies (ohne PyTorch)
!pip install -q -r requirements_colab.txt

# PyTorch mit CUDA Support für Colab
# Colab unterstützt normalerweise CUDA 11.8 oder 12.1
# Wir verwenden CUDA 11.8 als Standard (funktioniert mit den meisten Colab GPUs)
print("\n🔥 Installiere PyTorch mit CUDA Support...")
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118

print("\n✅ Installation abgeschlossen!")

## Repository Setup

Wählen Sie eine der folgenden Optionen:

**Option A**: Git Clone (empfohlen, wenn Repository auf GitHub/GitLab ist)
**Option B**: Upload (wenn Sie das Projekt lokal haben)

In [ ]:
# Option A: Git Clone
# Ersetzen Sie <repository_url> mit der URL Ihres Repositories
# Beispiel: https://github.com/username/Projekt-Archimedes.git

# !git clone <repository_url>
# %cd Projekt-Archimedes

# Option B: Upload (für lokale Entwicklung)
# Entkommentieren Sie die folgenden Zeilen und laden Sie das Projekt hoch

# from google.colab import files
# import zipfile
# import os
# 
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('.')
#         print(f"✅ {filename} entpackt")
# 
# # Finde den Projektordner
# project_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and 'archimedes' in d.lower()]
# if project_dirs:
#     %cd {project_dirs[0]}

# Für dieses Beispiel nehmen wir an, dass das Repository bereits geklont wurde
# Wenn Sie Option A oder B verwenden, kommentieren Sie die folgende Zeile aus:
print("⚠️ Bitte wählen Sie Option A oder B oben aus und führen Sie diese Zelle erneut aus!")

In [ ]:
# Zelle 3: System-Check
# Überprüft GPU-Verfügbarkeit und CUDA-Version

import torch
import sys

print("=" * 60)
print("SYSTEM INFORMATION")
print("=" * 60)

print(f"Python Version: {sys.version}")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA verfügbar: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"\n🎮 GPU Information:")
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA Version: {torch.version.cuda}")
    print(f"   cuDNN Version: {torch.backends.cudnn.version()}")
    
    props = torch.cuda.get_device_properties(0)
    print(f"   GPU Speicher: {props.total_memory / 1024**3:.2f} GB")
    print(f"   Compute Capability: {props.major}.{props.minor}")
    print(f"   Multiprocessors: {props.multi_processor_count}")
else:
    print("\n⚠️ Keine GPU gefunden!")
    print("   Bitte aktivieren Sie GPU: Laufzeit → Laufzeittyp ändern → GPU")

print("=" * 60)

## Optional: System Benchmark

Führen Sie diese Zelle aus, um Ihre Hardware zu benchmarken und optimale Trainingsparameter zu erhalten.

**Hinweis**: Der Benchmark kann einige Minuten dauern. Sie können ihn überspringen und direkt mit dem Training beginnen (die Skripte haben bereits gute Defaults für Colab).

In [ ]:
# Optional: System Benchmark
# Überspringt GPU-Tests für schnellere Ausführung (empfohlen für Colab)

!python benchmark_system.py --skip-gpu-test

print("\n✅ Benchmark abgeschlossen!")
print("Die Ergebnisse wurden in 'benchmark_results.json' gespeichert.")
print("Die Trainingsskripte verwenden diese automatisch mit --auto-config")

## Training starten

Wählen Sie eines der folgenden Trainingsskripte:

1. **End-to-End Training** (empfohlen): Trainiert TPN und SAN zusammen durch Self-Play
2. **TPN Training**: Trainiert nur das Tactical Perception Network
3. **SAN Training**: Trainiert nur das Strategic Abstraction Network

### End-to-End Training (Self-Play)

Dies ist die empfohlene Methode für das vollständige Training.

In [ ]:
# End-to-End Training mit automatischer Konfiguration
# --auto-config verwendet die Benchmark-Ergebnisse für optimale Parameter

# Parameter anpassen:
total_games = 100  # Anzahl der Self-Play-Spiele
training_iterations = 1000  # Anzahl der Training-Iterationen

# Starte Training
!python train_end_to_end.py \
    --auto-config \
    --total-games {total_games} \
    --training-iterations {training_iterations}

print("\n✅ Training abgeschlossen!")

### Alternative: TPN Training (nur Tactical Network)

Falls Sie nur das TPN trainieren möchten, benötigen Sie zuerst einen Datensatz.

In [ ]:
# TPN Training Beispiel
# Hinweis: Sie benötigen zuerst einen Datensatz mit create_dataset.py

# !python train_tpn.py \
#     --dataset-dir ./data/shards_tpn \
#     --auto-config \
#     --epochs 20

print("⚠️ Diese Zelle ist auskommentiert, da ein Datensatz benötigt wird.")
print("   Verwenden Sie create_dataset.py, um einen Datensatz zu erstellen.")

### Alternative: SAN Training (nur Strategic Network)

Falls Sie nur das SAN trainieren möchten, benötigen Sie zuerst einen Datensatz.

In [ ]:
# SAN Training Beispiel
# Hinweis: Sie benötigen zuerst einen Datensatz mit create_dataset.py

# !python train_san.py \
#     --dataset-dir ./data/shards_san \
#     --auto-config \
#     --epochs 20

print("⚠️ Diese Zelle ist auskommentiert, da ein Datensatz benötigt wird.")
print("   Verwenden Sie create_dataset.py, um einen Datensatz zu erstellen.")

## Model speichern und laden

Nach dem Training können Sie das Modell speichern und später wieder laden.

In [ ]:
# Beispiel: Model speichern
# (Dies sollte normalerweise im Trainingsskript integriert sein)

import torch
from src.archimedes.model import TPN, SAN, PlanToMoveMapper

# Beispiel-Code zum Speichern (anpassen nach Bedarf):
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tpn = TPN().to(device)
# san = SAN().to(device)
# mapper = PlanToMoveMapper().to(device)
# 
# # ... Training ...
# 
# # Speichern
# torch.save({
#     'tpn_state_dict': tpn.state_dict(),
#     'san_state_dict': san.state_dict(),
#     'mapper_state_dict': mapper.state_dict(),
# }, 'archimedes_model.pth')
# 
# print("✅ Model gespeichert als 'archimedes_model.pth'")

print("📝 Verwenden Sie diesen Code, um Modelle zu speichern/laden.")
print("   Passen Sie die Pfade und Variablennamen an Ihr Training an.")